In [ ]:
import re

import pandas as pd

In [ ]:
df = pd.read_csv("data/train_with_noise_ratio_1.csv")

### 추가로 처리해야할 샘플

In [ ]:
# normal에 있는 노이즈
in_clean_not_clean = [
    1095,
    2682,
    1483,
    1022,
    370,
    1161,
    2732,
    1554,
    1307,
    775,
    949,
    453,
    2492,
    116,
    1009,
    1608,
    131,
    895,
    1616,
    818,
    2442,
    1233,
    804,
    2648,
]
# noise에 있는 노말(클린)
in_noisy_not_noisy = [
    1027,
    736,
    746,
    1847,
    652,
    2434,
    1692,
    2111,
    1707,
    1688,
    2427,
    2009,
    2289,
    1438,
    2491,
    853,
    2139,
    957,
    2555,
    1994,
]

In [ ]:
print(len(in_clean_not_clean))
print(len(in_noisy_not_noisy))

24
20


#### 1차 필터링(룰 베이스)
in_noisy_not_noisy 를 clean data에 포함시킬 수 있다

In [ ]:
def noise_check_by_rule(text: str) -> bool:
    tokens = text.split()

    # 교차 횟수 계산 함수
    def count_crossings(token):
        crossing_count = 0
        previous_type = None

        # 토큰 길이가 2 미만인 경우는 노이즈로 판단하지 않음
        if len(token) < 2:
            return 0

        for char in token:
            # 한글, 숫자, 영문자, 특수문자로 타입 분류
            if "가" <= char <= "힣":  # 한글
                current_type = "K"
            elif "0" <= char <= "9":  # 숫자
                current_type = "N"
            elif "A" <= char <= "Z" or "a" <= char <= "z":  # 영문자
                current_type = "E"
            else:  # 특수문자
                current_type = "S"

            # 이전 타입과 다른 타입이 나오면 교차 발생
            if previous_type and previous_type != current_type:
                crossing_count += 1
            previous_type = current_type

        return crossing_count

    # 각 토큰에 대해 교차 횟수 검사
    for token in tokens:
        # 예외 처리 1: 한국어와 숫자가 붙어있는 경우는 괜찮음
        if re.fullmatch(r"[가-힣]+[0-9]+|[0-9]+[가-힣]+", token):
            continue

        # 예외 처리 2: "..."이 포함된 토큰은 노이즈로 간주하지 않음
        if "..." in token:
            continue

        # 교차가 2회 이상인 경우만 노이즈로 간주
        if count_crossings(token) >= 2:
            return True

    return False

In [ ]:
df["is_noised"] = df["text"].apply(noise_check_by_rule)

In [ ]:
df.head()

,ID,text,target,noise_ratio,noise_label,is_noised
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,0.450000,True,True
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,0.450000,True,True
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,0.357143,True,True
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0.000000,False,True
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,0.352941,True,True


기존 필터링보다 엄격한 기준이기 때문에 noise_label이 False인 샘플들이 is_noised = True인 경우 생김

확실한 노이즈 데이터를 걸러내는게 목적

In [ ]:
first_filtered = df[df["is_noised"] == True]
first_survived = df[df["is_noised"] == False]

In [ ]:
not_clean = first_survived[first_survived.index.isin(in_clean_not_clean)]

In [ ]:
not_clean

,ID,text,target,noise_ratio,noise_label,is_noised
1608,ynat-v1_train_01608,신yFdP 조) 왕실% 간절한 기도처,0,0.181818,False,False


In [ ]:
noise_candidate = first_survived[first_survived["noise_label"] == True]

In [ ]:
len(noise_candidate)

33

In [ ]:
not_noise = noise_candidate[noise_candidate.index.isin(in_noisy_not_noisy)]

In [ ]:
hidden_noise = noise_candidate[~noise_candidate.index.isin(in_noisy_not_noisy)]

In [ ]:
len(hidden_noise)

17

In [ ]:
hidden_noise

,ID,text,target,noise_ratio,noise_label,is_noised
115,ynat-v1_train_00115,최병륜 울산MBC 사장,4,0.250000,True,False
124,ynat-v1_train_00124,발언하\ 박광E,2,0.285714,True,False
184,ynat-v1_train_00184,프로농구 !울 SK 일` 교체 선수w L리엄스 영O,1,0.428571,True,False
330,ynat-v1_train_00330,KT Be Y 폰 2 단독 출시,3,0.428571,True,False
467,ynat-v1_train_00467,SKT유콘시스템 5G 드론용 ICT 솔루션 개발 협력,0,0.272727,True,False
543,ynat-v1_train_00543,구z 사이로 또 한 해가( 충남 해넘' 명소 북Z,0,0.266667,True,False
1181,ynat-v1_train_01181,jAIST ~I 월드i 8승,4,0.428571,True,False
1187,ynat-v1_train_01187,HDC 부동산114 자회사로 편입,6,0.200000,True,False
1444,ynat-v1_train_01444,SKT 갤럭시 A8 스타 단독 출시,3,0.285714,True,False
1980,ynat-v1_train_01980,게시판 ;로골퍼 박민m 필승UWS 펀드 1`,1,0.400000,True,False


근데 지금 보니까 진짜 노이즈 아닌 애들 발견

115, 467, 1187, 1444, 2299, 2302, 2321, 2421, 2612

In [ ]:
in_noisy_not_noisy_updated = [
    1027,
    736,
    746,
    1847,
    652,
    2434,
    1692,
    2111,
    1707,
    1688,
    2427,
    2009,
    2289,
    1438,
    2491,
    853,
    2139,
    957,
    2555,
    1994,
    115,
    467,
    1187,
    1444,
    2299,
    2302,
    2321,
    2421,
    2612,
]

In [ ]:
not_noise_updated = noise_candidate[noise_candidate.index.isin(in_noisy_not_noisy_updated)]

In [ ]:
len(not_noise_updated)

24

In [ ]:
hidden_noise_updated = noise_candidate[~noise_candidate.index.isin(in_noisy_not_noisy_updated)]

In [ ]:
hidden_noise_updated

,ID,text,target,noise_ratio,noise_label,is_noised
124,ynat-v1_train_00124,발언하\ 박광E,2,0.285714,True,False
184,ynat-v1_train_00184,프로농구 !울 SK 일` 교체 선수w L리엄스 영O,1,0.428571,True,False
330,ynat-v1_train_00330,KT Be Y 폰 2 단독 출시,3,0.428571,True,False
543,ynat-v1_train_00543,구z 사이로 또 한 해가( 충남 해넘' 명소 북Z,0,0.266667,True,False
1181,ynat-v1_train_01181,jAIST ~I 월드i 8승,4,0.428571,True,False
1980,ynat-v1_train_01980,게시판 ;로골퍼 박민m 필승UWS 펀드 1`,1,0.400000,True,False
2079,ynat-v1_train_02079,경희궁 옛 사진부w 100년전 놀이판eC f속박물관 기획7,0,0.214286,True,False
2638,ynat-v1_train_02638,w콩 T립 깃발,6,0.400000,True,False
2725,ynat-v1_train_02725,}성직렬 nC쇼,4,0.400000,True,False


In [ ]:
len(first_survived)
# 9개는 noise

603

In [ ]:
second_filtered = first_filtered[first_filtered["noise_label"] == True]
second_survived = first_filtered[first_filtered["noise_label"] == False]

In [ ]:
len(second_filtered)

1568

In [ ]:
len(second_survived)

629

In [ ]:
second_survived